In [1]:
from src.config import (
    GPT_KEY, CLAUDE_KEY, LLMAPI_KEY,
    GPT_MODEL, CLAUDE_MODEL, LLMAPI_MODEL,
    CSV_PATH, FIND_DIR, COMP_DIR,
    MAX_TOKENS, TEMPERATURE
)
from src.llm_clients import GPTClient, ClaudeClient, LlamaAPIClient
from src.schemas import ResolvedSentence, CompletionDoc, PartialResolved
from src.prompts import get_prompt
from src.json_utils import parse_or_fix
from src.utils import split_sents, ensure_dir_exists, start_timer, log_tokens, save_stats_to_csv, estimate_tokens

import pandas as pd
import json
import tqdm.auto as tqdm
from typing import List
from datetime import datetime

In [2]:
# Prepare output directory
ensure_dir_exists(COMP_DIR)

# Load Abstract
df_abs = pd.read_csv(CSV_PATH, usecols=[0, 1], header=0, names=["pmid", "abstract"])
ABS_CACHE = df_abs.set_index("pmid")["abstract"].to_dict()

# Index the finding sentence for phase one loading
def load_findings(tag: str) -> dict[str, set[int]]:
    out = {}
    with open(f"{FIND_DIR}/{tag}.jsonl", "r", encoding="utf-8") as fr:
        for line in fr:
            j = json.loads(line)
            out[j["pmid"]] = set(j["finding_ids"])
    return out

find_ids = {t: load_findings(t) for t in ["gpt4o", "claude", "llama"]}

# Initialize LLM Client
clients = {
    "gpt4o": GPTClient(model=GPT_MODEL, key=GPT_KEY),
    "claude": ClaudeClient(model=CLAUDE_MODEL, key=CLAUDE_KEY),
    "llama": LlamaAPIClient(model=LLMAPI_MODEL, key=LLMAPI_KEY)
}

In [3]:
# Load prompt
system_prompt, fewshot = get_prompt("completion")

def build_msgs(pmid: str, sid: int, sent: str, context: str) -> list[dict]:
    user = (
        f"Context:\n{context}\n\n"
        f"Sentence [{sid}]:\n{sent}\n\n"
        "Return JSON:"
    )
    return fewshot + [{"role": "user", "content": user}]

In [4]:
from datetime import datetime

def run_completion(model_tag: str):
    cli = clients[model_tag]
    out_path = f"{COMP_DIR}/{model_tag}.jsonl"
    open(out_path, "w", encoding="utf-8").close()

    start_timer(model_tag, "completion")

    for pmid_str, ids in tqdm.tqdm(find_ids[model_tag].items(), desc=f"{model_tag} completion"):
        pmid_int = int(pmid_str)
        if pmid_int not in ABS_CACHE:
            continue

        context = ABS_CACHE[pmid_int]
        sents = split_sents(context)
        completed = []

        for sid in sorted(ids):
            if sid < 0 or sid >= len(sents):
                continue

            msgs = build_msgs(pmid_str, sid, sents[sid], context)

            try:
                # Run and record API call
                raw = cli.run(msgs, task_id=f"{model_tag}:{pmid_str}:{sid}")

                # Estimate tokens manually (for Claude / LLaMA)
                prompt_text = " ".join(m['content'] for m in msgs)
                estimated_tokens = estimate_tokens(prompt_text + raw)
                log_tokens(model_tag, "completion", estimated_tokens)

                part_list = parse_or_fix(raw, cli, msgs, target_class=List[PartialResolved])

                for part in part_list:
                    completed.append(
                        ResolvedSentence(
                            id=part.id,
                            original=sents[part.id] if part.id < len(sents) else sents[sid],
                            resolved=part.resolved
                        )
                    )

            except Exception as e:
                print(f"[{model_tag}][{pmid_str}][{sid}] failed → {e}")
                continue

        if completed:
            record = CompletionDoc(pmid=pmid_str, sentences=completed)
            with open(out_path, "a", encoding="utf-8") as fw:
                fw.write(json.dumps(record.model_dump(), ensure_ascii=False) + "\n")

    save_stats_to_csv("completion")
    print(f"{model_tag} sentence complete -> {out_path}")

In [5]:
# Run models
run_completion("gpt4o")

gpt4o completion:   0%|          | 0/100 [00:00<?, ?it/s]

[Retry 1] Invalid JSON: ValidationError – 2 validation errors for list[PartialResolved]
0.id
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
0.resolved
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
-> Raw output: {} 
[Retry 2] Invalid JSON: ValidationError – 2 validation errors for list[PartialResolved]
0.id
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
0.resolved
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
-> Raw output: {} 
[Retry 3] Invalid JSON: ValidationError – 2 validation errors for list[PartialResolved]
0.id
  Field required [type=missing, input_value={}, input_type=dict]
    For further information 

In [6]:
run_completion("claude")

claude completion:   0%|          | 0/100 [00:00<?, ?it/s]

[Retry 1] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 2] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 3] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[claude][33095090][3] failed → Final JSON parse failed: No valid JSON object or list found. Sample: [}...
[Retry 1] Invalid JSON: JSONDecodeError – Unterminated string starting at: line 1 column 24 (char 23)
-> Raw output: [{"id": 2, "resolved": "Eighty-six percent of spontaneous splenic arteriovenous fistulas occur in women.}, {"id": 2, "resolved": "Fifty-five percent of spontaneous splenic arteriovenous fistulas are associated with a preexisting splenic artery aneurysm.}] 
claude sentence complete -> completion/claude.jsonl


In [7]:
run_completion("llama")

llama completion:   0%|          | 0/100 [00:00<?, ?it/s]

[Retry 1] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 2] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 3] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[llama][27168519][5] failed → Final JSON parse failed: No valid JSON object or list found. Sample: [}...
[Retry 1] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 2] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 3] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[llama][27168519][17] failed → Final JSON parse failed: No valid JSON object or list found. Sample: [}...
[Retry 1] Invalid JSON: ValueError – No valid JSON object or list found. Sample: [}...
-> Raw output: [] 
[Retry 2] Invalid JSON: ValueError – No valid J